# मानव-इन-द-लूप वर्कफ्लो Microsoft Agent Framework संग

## 🎯 सिक्ने उद्देश्यहरू

यस नोटबुकमा, तपाईं Microsoft Agent Framework को `RequestInfoExecutor` प्रयोग गरेर **मानव-इन-द-लूप** वर्कफ्लो कार्यान्वयन गर्न सिक्नुहुनेछ। यो शक्तिशाली ढाँचा AI वर्कफ्लोहरूलाई रोक्न र मानव इनपुट सङ्कलन गर्न अनुमति दिन्छ, जसले तपाईंको एजेन्टहरूलाई अन्तरक्रियात्मक बनाउँछ र महत्त्वपूर्ण निर्णयहरूमा मानवलाई नियन्त्रण दिन्छ।

## 🔄 मानव-इन-द-लूप के हो?

**मानव-इन-द-लूप (HITL)** एउटा डिजाइन ढाँचा हो जहाँ AI एजेन्टहरूले कार्यान्वयन रोक्छन् र अगाडि बढ्नु अघि मानव इनपुटको अनुरोध गर्छन्। यो निम्नका लागि आवश्यक छ:

- ✅ **महत्त्वपूर्ण निर्णयहरू** - महत्त्वपूर्ण कार्यहरू अघि मानव अनुमोदन प्राप्त गर्नुहोस्
- ✅ **अस्पष्ट परिस्थितिहरू** - AI अनिश्चित हुँदा मानवलाई स्पष्ट गर्न दिनुहोस्
- ✅ **प्रयोगकर्ता प्राथमिकताहरू** - धेरै विकल्पहरू बीच प्रयोगकर्तालाई छनोट गर्न दिनुहोस्
- ✅ **अनुपालन र सुरक्षा** - नियमन गरिएको कार्यहरूमा मानव निरीक्षण सुनिश्चित गर्नुहोस्
- ✅ **अन्तरक्रियात्मक अनुभवहरू** - प्रयोगकर्ताको इनपुटमा प्रतिक्रिया दिने संवादात्मक एजेन्टहरू निर्माण गर्नुहोस्

## 🏗️ Microsoft Agent Framework मा यो कसरी काम गर्छ

फ्रेमवर्कले HITL का लागि तीन प्रमुख घटकहरू प्रदान गर्दछ:

1. **`RequestInfoExecutor`** - विशेष कार्यान्वयनकर्ता जसले वर्कफ्लो रोक्छ र `RequestInfoEvent` उत्पन्न गर्छ
2. **`RequestInfoMessage`** - मानवलाई पठाइने टाइप गरिएको अनुरोध पेलोडहरूको आधार वर्ग
3. **`RequestResponse`** - `request_id` प्रयोग गरेर मानव प्रतिक्रियाहरूलाई मूल अनुरोधहरूसँग सम्बन्धित बनाउँछ

**वर्कफ्लो ढाँचा:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 हाम्रो उदाहरण: प्रयोगकर्ता पुष्टि सहित होटल बुकिङ

हामी सशर्त वर्कफ्लोमा आधारित भएर वैकल्पिक गन्तव्यहरू सुझाव दिनु अघि मानव पुष्टि थप्नेछौं:

1. प्रयोगकर्ताले गन्तव्य अनुरोध गर्छ (जस्तै, "पेरिस")
2. `availability_agent` ले कोठाहरू उपलब्ध छन् कि छैनन् जाँच गर्छ
3. **यदि कोठा उपलब्ध छैन** → `confirmation_agent` ले सोध्छ "तपाईं वैकल्पिक गन्तव्यहरू हेर्न चाहनुहुन्छ?"
4. वर्कफ्लो **`RequestInfoExecutor` प्रयोग गरेर रोक्छ**
5. **मानव प्रतिक्रिया दिन्छ** "हो" वा "होइन" कन्सोल इनपुट मार्फत
6. `decision_manager` प्रतिक्रिया अनुसार मार्गनिर्देशन गर्छ:
   - **हो** → वैकल्पिक गन्तव्यहरू देखाउनुहोस्
   - **होइन** → बुकिङ अनुरोध रद्द गर्नुहोस्
7. अन्तिम परिणाम देखाउनुहोस्

यसले प्रयोगकर्तालाई एजेन्टको सुझावहरूमा नियन्त्रण दिन कसरी सक्षम बनाउने देखाउँछ!

---

सुरु गरौं! 🚀


## चरण १: आवश्यक लाइब्रेरीहरू आयात गर्नुहोस्

हामी मानक Agent Framework का कम्पोनेन्टहरू आयात गर्छौं साथै **मानव-इन-द-लूप विशेष कक्षाहरू**:
- `RequestInfoExecutor` - मानव इनपुटको लागि कार्यप्रवाह रोक्ने कार्यकारी
- `RequestInfoEvent` - मानव इनपुट अनुरोध गर्दा उत्पन्न हुने घटना
- `RequestInfoMessage` - टाइप गरिएको अनुरोध पेलोडहरूको आधार कक्षा
- `RequestResponse` - अनुरोधहरूसँग मानव प्रतिक्रियाहरूलाई सम्बन्धित बनाउने
- `WorkflowOutputEvent` - कार्यप्रवाहको आउटपुट पत्ता लगाउनको लागि घटना


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## चरण २: संरचित आउटपुटहरूको लागि Pydantic मोडेलहरू परिभाषित गर्नुहोस्

यी मोडेलहरूले एजेन्टहरूले फिर्ता गर्ने **schema** परिभाषित गर्छन्। हामी सशर्त कार्यप्रवाहबाट सबै मोडेलहरू राख्छौं र थप्छौं:

**मानव-इन-द-लूपको लागि नयाँ:**
- `HumanFeedbackRequest` - `RequestInfoMessage` को उपवर्ग जसले मानवहरूलाई पठाइने अनुरोध पेलोड परिभाषित गर्छ
  - यसमा `prompt` (सोधिने प्रश्न) र `destination` (उपलब्ध नभएको सहरको सन्दर्भ) समावेश छ।


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## चरण ३: होटल बुकिङ उपकरण बनाउनुहोस्

उही उपकरण सशर्त कार्यप्रवाहबाट - गन्तव्यमा कोठाहरू उपलब्ध छन् कि छैनन् भनेर जाँच गर्दछ।


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## चरण ४: रुटिङका लागि सर्त कार्यहरू परिभाषित गर्नुहोस्

हामीलाई हाम्रो मानव-इन-द-लूप वर्कफ्लोका लागि **चार सर्त कार्यहरू** आवश्यक छ:

**सर्त वर्कफ्लोबाट:**
1. `has_availability_condition` - होटलहरू उपलब्ध हुँदा रुट गर्दछ
2. `no_availability_condition` - होटलहरू उपलब्ध नभएको अवस्थामा रुट गर्दछ

**मानव-इन-द-लूपका लागि नयाँ:**
3. `user_wants_alternatives_condition` - प्रयोगकर्ताले विकल्पहरू चाहन्छ भने रुट गर्दछ
4. `user_declines_alternatives_condition` - प्रयोगकर्ताले विकल्पहरू अस्वीकार गरेमा रुट गर्दछ


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorResponse from decision_manager to see if user said 'yes'.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        # The decision_manager yields a simple text response
        response_text = message.agent_run_response.text.lower().strip()
        
        display(
            HTML(f"""
            <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{response_text == 'yes'}</strong>
            </div>
        """)
        )
        
        return response_text == "yes"
    except Exception as e:
        return False


def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorResponse from decision_manager to see if user said 'no'.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        response_text = message.agent_run_response.text.lower().strip()
        
        display(
            HTML(f"""
            <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{response_text == 'no'}</strong>
            </div>
        """)
        )
        
        return response_text == "no"
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## चरण ५: निर्णय प्रबन्धक कार्यान्वयनकर्ता बनाउनुहोस्

यो **मानव-इन-द-लूप ढाँचाको मुख्य भाग** हो! `DecisionManager` एक अनुकूलित `Executor` हो जसले:

1. **मानव प्रतिक्रिया प्राप्त गर्छ** `RequestResponse` वस्तुहरू मार्फत
2. **प्रयोगकर्ताको निर्णय प्रक्रिया गर्छ** (हो/होइन)
3. **कार्यप्रवाहलाई मार्गदर्शन गर्छ** उपयुक्त एजेन्टहरूलाई सन्देश पठाएर

मुख्य विशेषताहरू:
- `@handler` डेकोरेटर प्रयोग गरेर विधिहरूलाई कार्यप्रवाह चरणको रूपमा सार्वजनिक गर्छ
- `RequestResponse[HumanFeedbackRequest, str]` प्राप्त गर्छ जसमा मूल अनुरोध र प्रयोगकर्ताको उत्तर समावेश हुन्छ
- साधारण "हो" वा "होइन" सन्देशहरू उत्पादन गर्छ जसले हाम्रो सर्त कार्यहरूलाई ट्रिगर गर्छ


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and route workflow accordingly.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        We send a simple message that will be caught by our condition functions.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            # User wants alternatives - send message to trigger alternative_agent
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Sending to alternative_agent
                </div>
            """)
            )
            # Send a message requesting alternatives for the destination
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            # User declined - send message to trigger cancellation
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Sending cancellation message
                </div>
            """)
            )
            # Send a simple "no" message that will be caught by the condition
            user_msg = ChatMessage(
                Role.USER,
                text="no",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(Role.USER, text="no")
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## चरण ६: कस्टम डिस्प्ले एक्जिक्युटर बनाउनुहोस्

उही डिस्प्ले एक्जिक्युटर सशर्त वर्कफ्लोबाट - अन्तिम नतिजा वर्कफ्लो आउटपुटको रूपमा प्रदान गर्दछ।


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## चरण ७: वातावरणीय भेरिएबलहरू लोड गर्नुहोस्

LLM क्लाइन्ट (GitHub Models, Azure OpenAI, वा OpenAI) कन्फिगर गर्नुहोस्।


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## चरण ८: एआई एजेन्टहरू र कार्यान्वयनकर्ता बनाउनुहोस्

हामी **छवटा कार्यप्रवाह घटकहरू** बनाउँछौं:

**एजेन्टहरू (AgentExecutor मा समेटिएका):**
1. **availability_agent** - उपकरण प्रयोग गरेर होटलको उपलब्धता जाँच गर्छ
2. **confirmation_agent** - 🆕 मानव पुष्टि अनुरोध तयार गर्छ
3. **alternative_agent** - वैकल्पिक शहरहरूको सुझाव दिन्छ (जब प्रयोगकर्ताले हो भन्छ)
4. **booking_agent** - बुकिङको प्रोत्साहन दिन्छ (जब कोठाहरू उपलब्ध छन्)
5. **cancellation_agent** - 🆕 रद्द गर्ने सन्देशको व्यवस्थापन गर्छ (जब प्रयोगकर्ताले होइन भन्छ)

**विशेष कार्यान्वयनकर्ता:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` जसले मानव इनपुटको लागि कार्यप्रवाहलाई रोक्छ
7. **decision_manager** - 🆕 कस्टम कार्यान्वयनकर्ता जसले मानव प्रतिक्रियाको आधारमा मार्गनिर्देशन गर्छ (पहिले नै माथि परिभाषित गरिएको)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## चरण ९: मानव-इन-द-लूप सहित कार्यप्रवाह निर्माण गर्नुहोस्

अब हामी **सशर्त मार्गनिर्देशन** सहित मानव-इन-द-लूप मार्ग समावेश गरेर कार्यप्रवाह ग्राफ निर्माण गर्छौं:

**कार्यप्रवाह संरचना:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**मुख्य किनाराहरू:**
- `availability_agent → confirmation_agent` (जब कोठा उपलब्ध छैन)
- `confirmation_agent → prepare_human_request` (प्रकार परिवर्तन गर्नुहोस्)
- `prepare_human_request → request_info_executor` (मानवको लागि रोक)
- `request_info_executor → decision_manager` (सधैं - RequestResponse प्रदान गर्दछ)
- `decision_manager → alternative_agent` (जब प्रयोगकर्ताले "हो" भन्छ)
- `decision_manager → cancellation_agent` (जब प्रयोगकर्ताले "होइन" भन्छ)
- `availability_agent → booking_agent` (जब कोठा उपलब्ध छ)
- सबै मार्गहरू `display_result` मा समाप्त हुन्छन्


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## चरण १०: टेस्ट केस १ चलाउनुहोस् - उपलब्धता बिना सहर (पेरिस मानव पुष्टि सहित)

यो परीक्षणले **मानव-इन-द-लूपको पूर्ण चक्र** प्रदर्शन गर्दछ:

1. पेरिसमा होटलको अनुरोध गर्नुहोस्
2. availability_agent जाँच गर्छ → कोठा उपलब्ध छैन
3. confirmation_agent मानवलाई सोध्ने प्रश्न तयार गर्छ
4. request_info_executor **वर्कफ्लो रोक्छ** र `RequestInfoEvent` निकाल्छ
5. **एप्लिकेसनले घटना पत्ता लगाउँछ र प्रयोगकर्तालाई कन्सोलमा सोध्छ**
6. प्रयोगकर्ताले "हो" वा "होइन" टाइप गर्छ
7. एप्लिकेसनले `send_responses_streaming()` मार्फत प्रतिक्रिया पठाउँछ
8. decision_manager प्रतिक्रिया अनुसार मार्ग निर्धारण गर्छ
9. अन्तिम नतिजा देखाइन्छ

**मुख्य ढाँचा:**
- पहिलो पटकको लागि `workflow.run_stream()` प्रयोग गर्नुहोस्
- अर्को पटकको लागि `workflow.send_responses_streaming(pending_responses)` प्रयोग गर्नुहोस्
- मानव इनपुट आवश्यक हुँदा `RequestInfoEvent` सुन्नुहोस्
- अन्तिम नतिजा पत्ता लगाउन `WorkflowOutputEvent` सुन्नुहोस्


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## चरण ११: परीक्षण केस २ चलाउनुहोस् - उपलब्धता भएको सहर (स्टकहोम - मानव इनपुट आवश्यक छैन)

यो परीक्षणले कोठा उपलब्ध हुँदा **सिधा मार्ग** देखाउँछ:

1. स्टकहोममा होटलको अनुरोध गर्नुहोस्
2. availability_agent जाँच गर्छ → कोठा उपलब्ध छन् ✅
3. booking_agent बुकिङको सिफारिस गर्छ
4. display_result पुष्टि देखाउँछ
5. **मानव इनपुट आवश्यक छैन!**

जब कोठा उपलब्ध हुन्छन्, कार्यप्रवाहले मानव-इन-द-लूप मार्गलाई पूर्ण रूपमा छोड्छ।


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## मुख्य बुँदाहरू र मानव-इन-द-लूप उत्कृष्ट अभ्यासहरू

### ✅ तपाईंले के सिक्नुभयो:

#### 1. **RequestInfoExecutor ढाँचा**
Microsoft Agent Framework मा मानव-इन-द-लूप ढाँचाले तीन मुख्य घटकहरू प्रयोग गर्दछ:
- `RequestInfoExecutor` - कार्यप्रवाह रोक्छ र घटनाहरू उत्सर्जन गर्दछ
- `RequestInfoMessage` - टाइप गरिएको अनुरोध पेलोडहरूको आधार वर्ग (यसलाई उपवर्ग बनाउनुहोस्!)
- `RequestResponse` - मानव प्रतिक्रियाहरूलाई मूल अनुरोधहरूसँग सम्बन्धित गर्दछ

**महत्वपूर्ण बुझाइ:**
- `RequestInfoExecutor` आफैं इनपुट सङ्कलन गर्दैन - यसले केवल कार्यप्रवाह रोक्छ
- तपाईंको एप्लिकेसन कोडले `RequestInfoEvent` सुन्नु पर्छ र इनपुट सङ्कलन गर्नुपर्छ
- तपाईंले `send_responses_streaming()` लाई `request_id` लाई प्रयोगकर्ताको उत्तरसँग नक्सा गर्ने dict को साथ कल गर्नुपर्छ

#### 2. **स्ट्रीमिङ कार्यप्रवाह ढाँचा**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **इभेन्ट-ड्राइभन आर्किटेक्चर**
विशेष घटनाहरू सुन्नुहोस् र कार्यप्रवाह नियन्त्रण गर्नुहोस्:
- `RequestInfoEvent` - मानव इनपुट आवश्यक छ (कार्यप्रवाह रोकियो)
- `WorkflowOutputEvent` - अन्तिम नतिजा उपलब्ध छ (कार्यप्रवाह पूरा भयो)
- `WorkflowStatusEvent` - अवस्था परिवर्तनहरू (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, आदि)

#### 4. **@handler प्रयोग गरेर कस्टम Executors**
`DecisionManager` ले देखाउँछ कि कसरी Executors सिर्जना गर्ने:
- `@handler` डेकोरेटर प्रयोग गरेर विधिहरूलाई कार्यप्रवाह चरणहरूका रूपमा उजागर गर्नुहोस्
- टाइप गरिएको सन्देशहरू प्राप्त गर्नुहोस् (जस्तै, `RequestResponse[HumanFeedbackRequest, str]`)
- कार्यप्रवाहलाई अन्य Executors मा सन्देशहरू पठाएर मार्गदर्शन गर्नुहोस्
- `WorkflowContext` मार्फत सन्दर्भ पहुँच गर्नुहोस्

#### 5. **मानव निर्णयहरूसँग सशर्त मार्गदर्शन**
तपाईं मानव प्रतिक्रियाहरू मूल्याङ्कन गर्ने सर्त कार्यहरू सिर्जना गर्न सक्नुहुन्छ:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 वास्तविक-विश्व अनुप्रयोगहरू:

1. **अनुमोदन कार्यप्रवाहहरू**
   - खर्च रिपोर्टहरू प्रशोधन गर्नु अघि प्रबन्धक अनुमोदन प्राप्त गर्नुहोस्
   - स्वचालित इमेलहरू पठाउनु अघि मानव समीक्षा आवश्यक छ
   - उच्च-मूल्य लेनदेनहरू कार्यान्वयन गर्नु अघि पुष्टि गर्नुहोस्

2. **सामग्री मोडरेशन**
   - शङ्कास्पद सामग्रीलाई मानव समीक्षाको लागि फ्ल्याग गर्नुहोस्
   - किनारा केसहरूमा अन्तिम निर्णय लिन मोडरेटरहरूलाई सोध्नुहोस्
   - जब AI को आत्मविश्वास कम छ, मानवमा बढाउनुहोस्

3. **ग्राहक सेवा**
   - AI लाई नियमित प्रश्नहरू स्वचालित रूपमा ह्यान्डल गर्न दिनुहोस्
   - जटिल मुद्दाहरूलाई मानव एजेन्टहरूमा बढाउनुहोस्
   - ग्राहकलाई सोध्नुहोस् कि उनीहरू मानवसँग कुरा गर्न चाहन्छन् कि चाहँदैनन्

4. **डाटा प्रशोधन**
   - अस्पष्ट डाटा प्रविष्टिहरू समाधान गर्न मानवलाई सोध्नुहोस्
   - अस्पष्ट कागजातहरूको AI व्याख्यालाई पुष्टि गर्नुहोस्
   - धेरै वैध व्याख्याहरू बीच प्रयोगकर्ताहरूलाई छनोट गर्न दिनुहोस्

5. **सुरक्षा-सम्बन्धित प्रणालीहरू**
   - अपरिवर्तनीय कार्यहरू अघि मानव पुष्टि आवश्यक छ
   - संवेदनशील डाटा पहुँच गर्नु अघि अनुमोदन प्राप्त गर्नुहोस्
   - नियमन गरिएका उद्योगहरूमा निर्णयहरू पुष्टि गर्नुहोस् (स्वास्थ्य सेवा, वित्त)

6. **इन्टरएक्टिभ एजेन्टहरू**
   - अनुवर्ती प्रश्नहरू सोध्ने संवादात्मक बोटहरू निर्माण गर्नुहोस्
   - प्रयोगकर्ताहरूलाई जटिल प्रक्रियाहरू मार्फत मार्गदर्शन गर्ने विजार्डहरू सिर्जना गर्नुहोस्
   - मानवहरूसँग चरण-दर-चरण सहयोग गर्ने एजेन्टहरू डिजाइन गर्नुहोस्

### 🔄 तुलना: मानव-इन-द-लूपको साथ र बिना

| विशेषता | सशर्त कार्यप्रवाह | मानव-इन-द-लूप कार्यप्रवाह |
|---------|---------------------|---------------------------|
| **कार्यान्वयन** | एकल `workflow.run()` | `run_stream()` + `send_responses_streaming()` को साथ लूप |
| **प्रयोगकर्ता इनपुट** | कुनै पनि छैन (पूर्ण रूपमा स्वचालित) | `input()` वा UI मार्फत अन्तरक्रियात्मक संकेतहरू |
| **घटकहरू** | एजेन्टहरू + Executors | + RequestInfoExecutor + DecisionManager |
| **घटनाहरू** | केवल AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, आदि |
| **रोक्ने** | रोक्ने छैन | कार्यप्रवाह RequestInfoExecutor मा रोक्छ |
| **मानव नियन्त्रण** | कुनै मानव नियन्त्रण छैन | मानवहरूले मुख्य निर्णयहरू लिन्छन् |
| **प्रयोग केस** | स्वचालन | सहयोग र निरीक्षण |

### 🚀 उन्नत ढाँचाहरू:

#### धेरै मानव निर्णय बिन्दुहरू
तपाईं एउटै कार्यप्रवाहमा धेरै `RequestInfoExecutor` नोडहरू राख्न सक्नुहुन्छ:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### टाइमआउट ह्यान्डलिङ
मानव प्रतिक्रियाहरूको लागि टाइमआउट कार्यान्वयन गर्नुहोस्:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### समृद्ध UI एकीकरण
`input()` को सट्टा, वेब UI, Slack, Teams, आदि संग एकीकृत गर्नुहोस्:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### सशर्त मानव-इन-द-लूप
विशेष परिस्थितिहरूमा मात्र मानव इनपुटको लागि सोध्नुहोस्:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ उत्कृष्ट अभ्यासहरू:

1. **सधैं RequestInfoMessage उपवर्ग बनाउनुहोस्**
   - प्रकार सुरक्षा र मान्यता प्रदान गर्दछ
   - UI रेंडरिङको लागि समृद्ध सन्दर्भ सक्षम गर्दछ
   - प्रत्येक अनुरोध प्रकारको उद्देश्य स्पष्ट गर्दछ

2. **वर्णनात्मक संकेतहरू प्रयोग गर्नुहोस्**
   - तपाईं के सोध्दै हुनुहुन्छ भन्ने बारे सन्दर्भ समावेश गर्नुहोस्
   - प्रत्येक विकल्पको परिणामहरू व्याख्या गर्नुहोस्
   - प्रश्नहरू सरल र स्पष्ट राख्नुहोस्

3. **अप्रत्याशित इनपुट ह्यान्डल गर्नुहोस्**
   - प्रयोगकर्ता प्रतिक्रियाहरू मान्य गर्नुहोस्
   - अमान्य इनपुटको लागि डिफल्टहरू प्रदान गर्नुहोस्
   - स्पष्ट त्रुटि सन्देशहरू दिनुहोस्

4. **अनुरोध IDs ट्र्याक गर्नुहोस्**
   - request_id र प्रतिक्रियाहरू बीचको सम्बन्ध प्रयोग गर्नुहोस्
   - राज्यलाई म्यानुअली व्यवस्थापन गर्ने प्रयास नगर्नुहोस्

5. **नन-ब्लकिङको लागि डिजाइन गर्नुहोस्**
   - इनपुटको प्रतीक्षा गर्दै थ्रेडहरू ब्लक नगर्नुहोस्
   - सम्पूर्णमा असिंक्रोनस ढाँचाहरू प्रयोग गर्नुहोस्
   - समवर्ती कार्यप्रवाह उदाहरणहरू समर्थन गर्नुहोस्

### 📚 सम्बन्धित अवधारणाहरू:

- **एजेन्ट मिडलवेयर** - एजेन्ट कलहरू अवरोध गर्नुहोस् (अघिल्लो नोटबुक)
- **कार्यप्रवाह अवस्था व्यवस्थापन** - रनहरू बीच कार्यप्रवाह अवस्था कायम राख्नुहोस्
- **बहु-एजेन्ट सहयोग** - मानव-इन-द-लूपलाई एजेन्ट टोलीहरूसँग संयोजन गर्नुहोस्
- **इभेन्ट-ड्राइभन आर्किटेक्चर** - घटनाहरूको साथ प्रतिक्रियात्मक प्रणालीहरू निर्माण गर्नुहोस्

---

### 🎓 बधाई छ!

तपाईंले Microsoft Agent Framework को साथ मानव-इन-द-लूप कार्यप्रवाहहरूमा महारत हासिल गर्नुभयो! तपाईं अब जान्नुहुन्छ:
- ✅ कार्यप्रवाहहरू रोक्न मानव इनपुट सङ्कलन गर्न
- ✅ RequestInfoExecutor र RequestInfoMessage प्रयोग गर्न
- ✅ घटनाहरूको साथ स्ट्रीमिङ कार्यान्वयन ह्यान्डल गर्न
- ✅ @handler को साथ कस्टम Executors सिर्जना गर्न
- ✅ मानव निर्णयहरूमा आधारित कार्यप्रवाह मार्गदर्शन गर्न
- ✅ मानवहरूसँग सहयोग गर्ने अन्तरक्रियात्मक AI एजेन्टहरू निर्माण गर्न

**यो विश्वसनीय, नियन्त्रणयोग्य AI प्रणालीहरू निर्माण गर्नको लागि एक महत्वपूर्ण ढाँचा हो!** 🚀



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरेर अनुवाद गरिएको हो। हामी यथासम्भव शुद्धता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छ। मूल दस्तावेज़ यसको मातृभाषामा आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुनेछैनौं।
